# Data exploration

This notebook will walk you through different ways we can look at data to gain a better understanding of our data, build intuition, and start hypothesis testing. By the end of this workshop, participants will have some data exploration skills in their toolbox to conduct which lays a solid foundation for any subsequent machine learning or data analysis projects.

It is assumed that you have two datafiles required stored in 'OUTPUT/' folder:
1. CQ_Data_20240602.txt
2. WtshdAttributeTable_20240602.txt

In [5]:
# Import all libraries at the top of your code so you can easily see and organize all the packges you are using. 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [6]:
# Set up the filepaths
outputDataFilepath ='C:/Users/danyk/Work/4_Data_Science/DataExplorationWorkshop/OUTPUT/'

In [22]:
CQ = pd.read_csv(outputDataFilepath+'CQ_Data_20240602.txt', sep=",", dtype={'SITE': str, 'HUC02': str})
WtshdAtt = pd.read_csv(outputDataFilepath+'WtshdAttributeTable_20240602.txt', sep=",", dtype={'SITE': str, 'HUC02': str})

WtshdAtt.head()

,SITE,YEAR,NAME,LAT,LONG,STATE,AREA_SQKM,HUC02,HCDN-2009,CLASS,...,UrbLU_pct,NatLU_pct,AgLU_pct,FERT_kgkm2,MANU_kgkm2,BFI_AVE,SLOPE_pct,AI,medianGWD_m,TD_pct
0,01011000,1980,"Allagash River near Allagash, Maine",47.069611,-69.079544,ME,3186.8,01,NaN,Non-ref,...,0.0,100.0,0.0,0.1,0.0,49.740415,7.61853,0.797182,10.467689,0.0
1,01011000,1981,"Allagash River near Allagash, Maine",47.069611,-69.079544,ME,3186.8,01,NaN,Non-ref,...,0.0,100.0,0.0,0.1,0.0,49.740415,7.61853,0.797182,10.467689,0.0
2,01011000,1982,"Allagash River near Allagash, Maine",47.069611,-69.079544,ME,3186.8,01,NaN,Non-ref,...,0.0,100.0,0.0,0.1,0.0,49.740415,7.61853,0.797182,10.467689,0.0
3,01011000,1983,"Allagash River near Allagash, Maine",47.069611,-69.079544,ME,3186.8,01,NaN,Non-ref,...,0.0,100.0,0.0,0.1,0.0,49.740415,7.61853,0.797182,10.467689,0.0
4,01011000,1984,"Allagash River near Allagash, Maine",47.069611,-69.079544,ME,3186.8,01,NaN,Non-ref,...,0.0,100.0,0.0,0.1,0.0,49.740415,7.61853,0.797182,10.467689,0.0
